In [86]:
import pandas as pd
accept = pd.read_csv('../rawData/accepted_2007_to_2018Q4.csv')

In [35]:
# These columns all have mixed datatypes
accept.columns[[0,19,49,59,118,129,130,131,134,135,136,139,145,146,147]]

Index(['id', 'desc', 'next_pymnt_d', 'verification_status_joint',
       'sec_app_earliest_cr_line', 'hardship_type', 'hardship_reason',
       'hardship_status', 'hardship_start_date', 'hardship_end_date',
       'payment_plan_start_date', 'hardship_loan_status',
       'debt_settlement_flag_date', 'settlement_status', 'settlement_date'],
      dtype='object')

In [87]:
# id is worth fixing
def find_weird(x):
    try:
        _ = int(x)
        return(True)
    except:
        return(False)

accept = accept[accept['id'].apply(find_weird)]
accept['id'] = accept['id'].apply(int)
accept['id'].dtype

In [91]:
pd.options.display.max_rows = 155
accept.sample(2).T

,1403264,454287
id,143983634,129557235
member_id,NaN,NaN
loan_amnt,10000,7900
funded_amnt,10000,7900
funded_amnt_inv,10000,7900
term,36 months,36 months
int_rate,8.19,6.71
installment,314.25,242.89
grade,A,A
sub_grade,A4,A3


In [13]:
accept.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Data columns (total 151 columns):
 #   Column                                      Non-Null Count    Dtype  
---  ------                                      --------------    -----  
 0   id                                          2260701 non-null  object 
 1   member_id                                   0 non-null        float64
 2   loan_amnt                                   2260668 non-null  float64
 3   funded_amnt                                 2260668 non-null  float64
 4   funded_amnt_inv                             2260668 non-null  float64
 5   term                                        2260668 non-null  object 
 6   int_rate                                    2260668 non-null  float64
 7   installment                                 2260668 non-null  float64
 8   grade                                       2260668 non-null  object 
 9   sub_grade                                   2260668 non-

In [15]:
accept['loan_status'].value_counts()

Fully Paid                                             1076751
Current                                                 878317
Charged Off                                             268559
Late (31-120 days)                                       21467
In Grace Period                                           8436
Late (16-30 days)                                         4349
Does not meet the credit policy. Status:Fully Paid        1988
Does not meet the credit policy. Status:Charged Off        761
Default                                                     40
Name: loan_status, dtype: int64

In [110]:
# Variation in annual issuance
import datetime as dt
def make_dateval(s):
    s = s.split('-')
    return(dt.datetime.strptime(s[1] + s[0] + '01', '%Y%b%d'))
    
#accept['issue_d'].str.split('-')
accept['issue_d'] = accept['issue_d'].apply(make_dateval)

In [113]:
accept.groupby('issue_d')['funded_amnt'].sum()

issue_d
2007-06-01        91850.0
2007-07-01       332100.0
2007-08-01       488600.0
2007-09-01       372950.0
2007-10-01       682875.0
2007-11-01       995950.0
2007-12-01      1827225.0
2008-01-01      2902250.0
2008-02-01      2951225.0
2008-03-01      4150050.0
2008-04-01      1811025.0
2008-05-01       582000.0
2008-06-01       654975.0
2008-07-01       809250.0
2008-08-01       468450.0
2008-09-01       270950.0
2008-10-01       945400.0
2008-11-01      2008675.0
2008-12-01      2420775.0
2009-01-01      2469700.0
2009-02-01      2708950.0
2009-03-01      3061300.0
2009-04-01      3071350.0
2009-05-01      3351250.0
2009-06-01      3591925.0
2009-07-01      3876125.0
2009-08-01      4316075.0
2009-09-01      5103275.0
2009-10-01      6289450.0
2009-11-01      6848875.0
2009-12-01      7126475.0
2010-01-01      7385950.0
2010-02-01      7456975.0
2010-03-01      8664750.0
2010-04-01      9510575.0
2010-05-01     10044250.0
2010-06-01     10038975.0
2010-07-01     11517600.0
2010

In [12]:
accept['purpose'].value_counts()

debt_consolidation    1277877
credit_card            516971
home_improvement       150457
other                  139440
major_purchase          50445
medical                 27488
small_business          24689
car                     24013
vacation                15525
moving                  15403
house                   14136
wedding                  2355
renewable_energy         1445
educational               424
Name: purpose, dtype: int64